<a href="https://colab.research.google.com/github/Limgyuho/-Create-a-calculator-using-the-static-method-without-creating-an-object./blob/master/%EA%B2%8C%EC%9E%84_%EC%A0%90%EC%88%98%EC%99%80_%EC%8B%9C%EA%B0%84_%ED%91%9C%EC%8B%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pygame
import random
import time
from datetime import datetime

# 1. 초기화 -> 변수 선언
pygame.init()
# 2. 게임 화면 설정 -> 크기 고정
size = [400, 650]
screen = pygame.display.set_mode(size)

title = "PyGameExam"
pygame.display.set_caption(title)

# 3. 게임 화면 내에서의 설정 -> 변수
clock = pygame.time.Clock()

black = (0, 0, 0)

#class로 묶기 
class Object:
    def __init__(self):
        self.x = 0
        self.y = 0
        self.move = 0
    #이미지가 png형식일경우와 아닐경우를 분리한다
    def add_img(self, address):
        if address[-3:] == "png":
            self.img = pygame.image.load(address).convert_alpha()
        else:
            self.img = pygame.image.load(address)
        self.width, self.height = self.img.get_size()
            
    #어떠한 이미지를 사용하더라도 사이즈가 자동적으로 변하게 해주기 위함이다
    def change_size(self, width, height):
        self.img = pygame.transform.scale(self.img, (width, height))
        self.width, self.height = self.img.get_size()
        
    def show(self):
        screen.blit(self.img, (self.x, self.y))
        
def crash(a, b):
    if (a.x-b.width <= b.x) and (b.x <= a.x+a.width):
        if (a.y-b.height <= b.y) and (b.y <= a.y+a.height):
            return True
        else:
            return False
    else : 
        return False



#클래스 실행문       
hero = Object()
#클래스 안에 hero에 이미지를 추가한다
hero.add_img("C:\\Users\\USER\\Pictures\\hero2.png")
#hero의 사이즈를 변경할수 있도록 지정해준다
hero.change_size(60,80)
hero.x = round(size[0] / 2) - round(hero.width / 2)
hero.y = size[1] - hero.height - 100
hero.move = 10

left_move = False
right_move = False
space_shot = False

kill = 0
loss = 0
Game_Over =0

missile_list = []
enemy_list = []

k = 0

#게임 시작 대기 화면
system_exit = 0
while(system_exit == 0):
    clock.tick(60)
    
    #게임이 시작 되기전의 상태이며 스페이스바를 눌렀을때 시작된다
    for event in pygame.event.get():
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_SPACE:
                system_exit = 1
        #맨위 for문 안에 있으며 게임이 시작되기전 안내 문구를 보여준다
        #게임 대기 화면은 검정색이다
        screen.fill(black)
        font = pygame.font.Font("C:\\Windows\\Fonts\\ariblk.ttf",15)
        text_kill = font.render("PRESS SPACEBAR TO START GAME",True,(255,255,255) )
        #대기 화면에서 안내 문구를 의 위치를 보여준다
        screen.blit(text_kill,(40,round(size[1]/2-50)))
        pygame.display.flip()
    
    

# 4. 메인이벤트

start_time = datetime.now()

system_exit = 0
while(system_exit == 0):
    #  4-1. FPS(Frame per Sec) 설정
    clock.tick(60)
    
    #  4-2. 입력(키보드, 마우스)의 감지

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            system_exit = 1
        #키다운을 계속 누를 경우
        #왼쪽으로 누르면 음의 방향으로
        #오른쪽으로 우르면 양의 방향으로 움직인다 
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_LEFT:
                left_move = True
            if event.key == pygame.K_RIGHT:
                right_move = True
            if event.key == pygame.K_SPACE:
                space_shot = True
        #키업과 다운을 분류 해야 되는데 업부분이 if이면
        #빠져나가지 못하여 정상작동이 되지 않는다
        elif event.type == pygame.KEYUP:
            if event.key == pygame.K_LEFT:
                left_move = False
            if event.key == pygame.K_RIGHT:
                right_move = False
            if event.key == pygame.K_SPACE:
                space_shot = False
    

    #  4-3. 입력, 시간에 따른 변화
    
    now_time = datetime.now()
    delta_time = round((now_time - start_time).total_seconds())
    
    #위의 키다운 부분에서 왼쪽이냐 오른쪽이냐에 따라
    #아래의 코드가 실행되고 다시 위로 올라가며 무한 반복한다
    #키업을 하면 아래의 코드를 실행하지 않고 가만히 멈추게 된다
    if left_move == True:
        hero.x -= hero.move
        #x좌표의끝은 0이므로 그밑을 벗어나서는 안된다
        #아래의 코드가 위로 간다면 아예 움직이지못하고 넘어가거나
        #조금만 움직이고 아래로 넘어가기 때문이다
        if hero.x<=0:
            hero.x =0
    elif right_move == True:
        hero.x += hero.move 
        #x축을 기준으로 한이유는 통일성을 주기 위함이다
        #siz[]로 한이유는 매번 값이 변경될수 있는 경우를 상정해야 하기 때문이다
        if hero.x >= size[0] - hero.width:
            hero.x = size[0] - hero.width
    #발사의 속도를 저절하며 크기가 배수의 크기가 클수록 발사의 속도가 줄어든다
    if space_shot == True and k % 8 == 0:
        missile = Object()
        missile.add_img("C:\\Users\\USER\Pictures\\missile.png")
        missile.change_size(25, 40)
        missile.x = round(hero.x + hero.width / 2 - missile.width / 2)
        missile.y = hero.y - missile.height - 10
        missile.move = 3
        missile_list.append(missile)
     
    k += 1
    
    delete_list = []
    for i in range(len(missile_list)):
        m = missile_list[i]
        m.y -= m.move
        if m.y <= -m.height:
            delete_list.append(m)
            
    delete_list.reverse()
            
    for d in delete_list:  
        missile_list.remove(d)
        
    if random.random() > 0.98:
        #random.random함수는 숫자를 일정 범위에서 무작위로 추출해주는 기능을 준다
        #보통 0부터 1까지 이며 현재는 0.98
        enemy = Object()
        enemy.add_img("C:\\Users\\USER\\Pictures\\enemy1.png")
        enemy.change_size(40,40)
        enemy.x = random.randrange(0,size[0]-enemy.width-round(hero.width/2))
        #random.randrange()함수는 랜덤함수의 범위 만큼 사이즈를 지정해준다
        #0부터 가로의 길이에서 적의 크기 빼기 히어로의 크기만큼빼준다
        #미사일이 히어로의 중앙에서 발사되기때문이다
        enemy.y = 10
        #위를 기준으로 10만큼 아래의 방향에서 새성된다
        enemy.move = 1
        enemy_list.append(enemy)
        
    delete_list = []
    
    for i in range(len(enemy_list)):
        enemy = enemy_list[i]
        enemy.y += enemy.move
        #적이 아래오 떨어져야 하기 때문에 +가 되어야 한다
        if enemy.y >= size[1]:
            delete_list.append(i)
        #만일 사이즈 밖이라면 삭제공간으로 간다
    delete_list.reverse()
    
    #범위 밖에 있는 것들은 아래의 반복문을 통하여 삭제된다
    for j in delete_list:
        del enemy_list[j]
        #화면 밖으로 사라질 경우
        loss += 1
        
    dm_list = []
    de_list = []
    #충돌에 따른 변화
    #미사일 리스트 순회
    for i in range(len(missile_list)):
        #적리스트 순회
        for j in range(len(enemy_list)):
            #미사일리스트의 인덱스의 i번을 새로 미사일이라는 변수에
            #적리스트의 인덱스 j번을 새로 적이라는 변수에
            missile = missile_list[i]
            enemy = enemy_list[j]
            #만일 crash(missile,enemy)이 참이라면
            if crash(missile,enemy) == True:
                #각각의 리스트에 i와j를 추가한다
                #삭제되는 공간을 생성해준다
                dm_list.append(i)
                de_list.append(j)
    #같은 요소가 추가 될수 있기에 중복을 제거 해주어야 한다
    #set()의 의미는 중복을 제거해준다
    #list로 감싸는 이유는 set의 기능은
    #리스트가 아닌 딕셔너리로 변환해준다
    dm_list = list(set(dm_list))
    de_list = list(set(de_list))
    dm_list.reverse()
    de_list.reverse()
    
    try:
    #범위 밖에 있는 요소들을 삭제
        for dm in dm_list:
            del missile_list[dm]
        for da in de_list:
            del enemy_list[da]
        #파괴 될때마다 죽인다
            kill +=1
    except:
        pass

   
    #적이 히어로와 충돌하면 꺼지도록 만들기
    #적리스트를 순회후 충돌이 일어나면 종료한다
    for i in range(len(enemy_list)):
        a = enemy_list[i]
        if crash(a, hero) == True:
            system_exit = 1
            Game_Over = 1
          
    #  4-4. 전사작업(그리기)
    screen.fill(black)
    hero.show()
    for m in missile_list:
        m.show()
        
    for e in enemy_list:
        e.show()
    
    #텍스트 
    font = pygame.font.Font("C:\\Windows\\Fonts\\ariblk.ttf",20)
    text_kill = font.render("kill : {} loss : {}".format(kill,loss),True,(255,255,0) )
    screen.blit(text_kill,(10,5))
    
    text_time = font.render("time : {} ".format(delta_time),True,(255,255,255) )
    screen.blit(text_time,(size[0]-100,5))
    
    #  4-5. 업데이트
    pygame.display.flip()

# 5. 종료
#게임 오버인 경우 종료
while Game_Over == 1:
    clock.tick(60)
    
    #닫기 버튼을 눌르경우 종료 된다
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            Game_Over = 0
        
        screen.fill(black)
        font = pygame.font.Font("C:\\Windows\\Fonts\\ariblk.ttf",40)
        text_kill = font.render("GAEM OVER",True,(255,0,0) )
        screen.blit(text_kill,(80,round(size[1]/2-45)))
        pygame.display.flip()
    

pygame.quit()